In [20]:
from tqdm.auto import tqdm
tqdm.pandas()
import pandas as pd
import pandas as pd
import numpy as np
import json, pickle
from tqdm.auto import tqdm
tqdm.pandas()

from pandarallel import pandarallel
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, OkapiBM25Model
from gensim.similarities import SparseMatrixSimilarity
import numpy as np
pandarallel.initialize(progress_bar=True, use_memory_fs=False, nb_workers=10)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [22]:
CORPUS_PATH = "../../data/MEDICAL/corpus"
PROCESSED_PATH = "../processed"

In [23]:
df = pd.read_csv(f"{PROCESSED_PATH}/corpus_clean.csv")
df = df.fillna("NaN")

In [24]:
# inline text processing code

import re
import string
from nltk import word_tokenize as lib_tokenizer

dict_map = dict({})  
def word_tokenize(text):
    words = text.split() 
    words_norm = [] 
    for w in words: 
        if dict_map.get(w, None) is None: 
            dict_map[w] = ' '.join(lib_tokenizer(w)).replace('``', '"').replace("''", '"') 
        words_norm.append(dict_map[w]) 
    return words_norm 

def strip_context(text): 
    text = text.replace('\n', ' ') 
    text = re.sub(r'\s+', ' ', text) 
    text = text.strip() 
    return text

def post_process(x):
    x = " ".join(word_tokenize(strip_context(x))).strip()
    x = x.replace("\n"," ")
    x = "".join([i for i in x if i not in string.punctuation])
    return x

In [25]:
df['bm25_text'] = df['bm25_text'].apply(lambda x: x.lower()).parallel_apply(post_process)
corpus = [x.split() for x in df['bm25_text'].values]
dictionary = Dictionary(corpus)
bm25_model = OkapiBM25Model(dictionary=dictionary)
bm25_corpus = bm25_model[list(map(dictionary.doc2bow, corpus))]
bm25_index = SparseMatrixSimilarity(bm25_corpus, num_docs=len(corpus), num_terms=len(dictionary),normalize_queries=False, normalize_documents=False)
tfidf_model = TfidfModel(dictionary=dictionary, smartirs='bnn')  # Enforce binary weighting of queries
dictionary.save(f"{PROCESSED_PATH}/outputs/bm25/dict")
tfidf_model.save(f"{PROCESSED_PATH}/outputs/bm25/tfidf")
bm25_index.save(f"{PROCESSED_PATH}/outputs/bm25/bm25_index")